<a href="https://colab.research.google.com/github/itimes-digital/deep-learning-estudo/blob/main/gan_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install keras_adversarial

In [2]:
import keras
keras.__version__

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:458: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:459: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:460: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:461: FutureWarn

'2.1.2'

In [3]:
pip install keras==2.1.2

In [4]:
!pip install tensorflow==1.2

### O componente keras_adversarial funciona penas no Keras 2.1.2 e no tensorflow 1.2



In [5]:
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import InputLayer, Dense, Flatten, Reshape
from keras.regularizers import L1L2
from keras_adversarial import AdversarialModel, simple_gan, gan_targets
from keras_adversarial import AdversarialOptimizerSimultaneous, normal_latent_sampling

*pip install --updgrade keras, volta a versão atual.*

In [6]:
(previsores_treinamento, _), (_,_) = mnist.load_data()
previsores_treinamento.shape

11493376/11490434 [==============================] - 1s 0us/step


(60000, 28, 28)

In [7]:
previsores_treinamento = previsores_treinamento.astype('float32') / 255
previsores_treinamento

array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0.

# Criação das redes neurais

### Gerador

In [8]:
gerador = Sequential()

gerador.add(Dense(units = 500, 
                  input_dim = 100, 
                  activation = 'relu',
                  kernel_regularizer = L1L2(1e-5, 1e-5)))# 1e-5, 1e-5 definido na documentação 

gerador.add(Dense(units = 500, 
                  input_dim = 100, 
                  activation = 'relu',
                  kernel_regularizer = L1L2(1e-5, 1e-5)))

gerador.add(Dense(units = 784, 
                  activation = 'sigmoid', 
                  kernel_regularizer = L1L2(1e-5, 1e-5)))# 28 * 28

gerador.add(Reshape((28,28)))

### Discriminador

In [9]:
discriminador = Sequential()

discriminador.add(InputLayer(input_shape = (28,28))) # camada de entrada

discriminador.add(Flatten())

discriminador.add(Dense(units = 500, 
                        activation = 'relu', 
                        kernel_regularizer = L1L2(1e-5, 1e-5)))

discriminador.add(Dense(units = 500, 
                        activation = 'relu', 
                        kernel_regularizer = L1L2(1e-5, 1e-5)))

discriminador.add(Dense(units = 1, 
                        activation = 'sigmoid', 
                        kernel_regularizer = L1L2(1e-5, 1e-5)))

In [10]:
gan = simple_gan(gerador, discriminador, normal_latent_sampling((100,)))# valor igual ao input_dim do gerador

In [11]:
model = AdversarialModel(base_model = gan,
                         player_params = [gerador.trainable_weights,
                                          discriminador.trainable_weights])

In [12]:
model.adversarial_compile(adversarial_optimizer = AdversarialOptimizerSimultaneous,
                          player_optimizers = ['adam', 'adam'],
                          loss = 'binary_crossentropy')

In [ ]:
model.fit(x = previsores_treinamento, 
          y = gan_targets(60000), epochs = 1, batch_size = 256)